In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
# Load the dataset
df = pd.read_csv('snitchcasestudy.csv') 

In [ ]:
# Dataset Overview
print("\n1. Dataset Overview")
print("-" * 50)
print("\nFirst few rows of the dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nDataset Statistics:")
print(df.describe())

In [ ]:
# Data cleaning
df['WEEK'] = pd.to_datetime(df['WEEK'])
df = df.sort_values(['sku group', 'WEEK'])
df = df.dropna()

In [ ]:
# slopr distribution for dyamic thresholding
slope_map = {}
for sku, group in df.groupby('sku group'):
    X = (group['WEEK'] - group['WEEK'].min()).dt.days.values.reshape(-1, 1)
    y = group['SALES'].values
    if len(X) > 3:
        reg = LinearRegression().fit(X, y)
        slope_map[sku] = reg.coef_[0]

mean_slope = np.mean(list(slope_map.values()))
std_slope = np.std(list(slope_map.values()))
increasing_threshold = mean_slope + std_slope
decreasing_threshold = mean_slope - std_slope

# classification of SKUs
def categorize_sku(sku_df):
    if len(sku_df) < 4:
        return 'No pattern'

    # Trend
    X = (sku_df['WEEK'] - sku_df['WEEK'].min()).dt.days.values.reshape(-1, 1)
    y = sku_df['SALES'].values
    reg = LinearRegression().fit(X, y)
    trend_slope = reg.coef_[0]

    # Seasonality
    try:
        result = seasonal_decompose(sku_df.set_index('WEEK')['SALES'], model='additive', period=4)
        seasonal_strength = np.var(result.seasonal) / np.var(result.observed)
    except:
        seasonal_strength = 0

    # Classification logic
    if seasonal_strength > 0.2:
        return 'Seasonal'
    elif trend_slope > increasing_threshold:
        return 'Increasing'
    elif trend_slope < decreasing_threshold:
        return 'Decreasing'
    else:
        return 'No pattern'

# Categorize SKUs
sku_categories = {}
for sku, group in df.groupby('sku group'):
    sku_categories[sku] = categorize_sku(group)

df['Category'] = df['sku group'].map(sku_categories)

# Demand Dorecasting
def forecast_demand(sku_df, horizon=13):
    if len(sku_df) < 4:
        return np.mean(sku_df['SALES']) * np.ones(horizon)
    try:
        sales_series = sku_df.set_index('WEEK')['SALES'].resample('W').sum().fillna(0)
        if sku_categories.get(sku_df['sku group'].iloc[0], '') == 'Seasonal':
            model = ExponentialSmoothing(sales_series, seasonal='add', seasonal_periods=4)
            fit = model.fit()
            return fit.forecast(horizon)
        else:
            model = ARIMA(sales_series, order=(1, 1, 1))
            fit = model.fit()
            return fit.forecast(steps=horizon)
    except Exception as e:
        print(f"Forecast failed for {sku_df['sku group'].iloc[0]}: {e}")
        return np.mean(sku_df['SALES']) * np.ones(horizon)

# EOQ calculation
def calculate_eoq(demand, holding_cost=0.2, ordering_cost=50):
    annual_demand = demand * 365 / 90
    return 0 if annual_demand == 0 else np.sqrt((2 * annual_demand * ordering_cost) / holding_cost)

demand_horizon_weeks = 13
forecast_dates = pd.date_range(start=df['WEEK'].max() + timedelta(days=7), periods=demand_horizon_weeks, freq='W')

results = []
forecast_records = []

for sku, group in df.groupby('sku group'):
    forecast = forecast_demand(group, horizon=demand_horizon_weeks)
    avg_demand = np.mean(forecast)
    eoq = calculate_eoq(avg_demand)
    last_inventory = group.iloc[-1]['INVENTORY']
    last_sales = group.iloc[-1]['SALES']
    category = sku_categories.get(sku, 'No pattern')

    results.append({
        'SKU Group': sku,
        'Category': category,
        'Avg_90day_Demand': avg_demand,
        'EOQ': eoq,
        'Current_Inventory': last_inventory,
        'Last_Sales': last_sales,
    })

    for i, date in enumerate(forecast_dates):
        forecast_records.append({'SKU Group': sku, 'Week': date, 'Forecasted_Sales': forecast[i]})

In [ ]:
def extract_sku_number(sku):
    match = re.search(r'\d+', sku)
    return int(match.group()) if match else 0

# result dataFrames
results_df = pd.DataFrame(results)
results_df['SKU_Num'] = results_df['SKU Group'].apply(extract_sku_number)
results_df = results_df.sort_values('SKU_Num').drop('SKU_Num', axis=1)

forecast_df = pd.DataFrame(forecast_records)
pivot_df = forecast_df.pivot(index='Week', columns='SKU Group', values='Forecasted_Sales')
pivot_df.reset_index(inplace=True)

pivot_df.insert(0, 'Week_Num', range(1, len(pivot_df) + 1))

sorted_columns = ['Week_Num', 'Week'] + sorted(
    [col for col in pivot_df.columns if col not in ['Week_Num', 'Week']],
    key=lambda x: extract_sku_number(x)
)
forecast_df = pivot_df[sorted_columns]

forecast_df = forecast_df.rename(columns={'Week_Num': ''})

# Saving results to CSV fil
results_df.to_csv('sku_analysis_results.csv', index=False)
forecast_df.to_csv('sku_forecast_90day.csv', index=False)

# Sample forecasting plot
sample_sku = results_df.iloc[0]['SKU Group']
sample_data = df[df['sku group'] == sample_sku]
plt.figure(figsize=(12, 6))
plt.plot(sample_data['WEEK'], sample_data['SALES'], 'b-', label='Historical Sales')
plt.plot(forecast_df['Week'],
         forecast_df[sample_sku],
         'r--', label='Forecast')
plt.title(f'Sales Forecast for {sample_sku} ({sku_categories.get(sample_sku)})')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

# result preview
results_df.head(), forecast_df.head()